In [15]:
object1 = dl.Data_load()
object1.load_data(train_file_name, test_file_name)

train_data = object1.train_data
test_data = object1.test_data

train_data.shape, test_data.shape

((143620, 3), (48852, 3))

In [18]:
input_txt = train_data['document']
input_txt

0                                         아 더빙 진짜 짜증나네요 목소리
1                                흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나
2                                         너무재밓었다그래서보는것을추천한다
3                                 교도소 이야기구먼 솔직히 재미는 없다평점 조정
4         사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...
                                ...                        
143615                  디케이드 다음에 더블 다음에 오즈인데 더블은 조금밖에 안나오네요
143616                                      인간이 문제지 소는 뭔죄인가
143617                        이게 뭐요 한국인은 거들먹거리고 필리핀 혼혈은 착하다
143618                           청춘 영화의 최고봉방황과 우울했던 날들의 자화상
143619                             한국 영화 최초로 수간하는 내용이 담긴 영화
Name: document, Length: 143620, dtype: object

In [48]:
from torch.utils.data import Dataset

class NaverMovieDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.df = df
        self.tokenizer = tokenizer
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        # id, document, label
        tokenized_example = self.tokenizer(self.df.iloc[idx, 1])
        label = self.df.iloc[idx, 2]
        tokenized_example = {k: torch.tensor(v) for k, v in tokenized_example.items()}
        tokenized_example.update({"labels": torch.tensor(label, dtype=torch.long)})
        return tokenized_example

In [49]:
from transformers import AutoModelForSequenceClassification

MODEL_NAME = 'matthewburke/korean_sentiment'

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)

In [50]:
model

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(50135, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [51]:
train_set = NaverMovieDataset(train_data, tokenizer)
train_set[0]

{'input_ids': tensor([    2,  2431, 48502,  7997, 15775,  8028, 10855,     3]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1]),
 'labels': tensor(0)}

In [52]:
from transformers import DataCollatorWithPadding
from torch.utils.data import DataLoader

train_loader = DataLoader(
    train_set, 
    batch_size=2, 
    shuffle=False, 
    collate_fn=DataCollatorWithPadding(
        tokenizer=tokenizer, padding=True
    )
)

In [53]:
batch = next(iter(train_loader)) # for loop 돌려야됨 for batch in dataloader

In [54]:
batch

{'input_ids': tensor([[    2,  2431, 48502,  7997, 15775,  8028, 10855,     3,     0,     0,
             0,     0,     0,     0,     0,     0],
        [    2,  3976, 41905,  4333,  8201, 10586, 14470,  4123, 48857, 21225,
          9799, 17087,  4020,  2436,  8037,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([0, 1])}

In [57]:
out = model(**batch)
out['loss'].backward()

In [58]:
preds = torch.argmax(out['logits'], dim=1)
preds

tensor([0, 0])

In [60]:
train_data[0:2]

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1


In [23]:
tokenized_out = [tokenizer(sentence) for sentence in train_data['document']]

{'input_ids': [2, 2431, 48502, 7997, 15775, 8028, 10855, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [25]:
tokenized_out[0]['input_ids']

[2, 2431, 48502, 7997, 15775, 8028, 10855, 3]

In [31]:
# MAX_LEN = max([len(tokenized_out[i]['input_ids']) for i in range(len(tokenized_out))])
# MAX_LEN

103

In [32]:
# MAX_LEN = max([len(tokenized_out[i]['input_ids']) for i in range(len(tokenized_out))])
# input_ids = [tokenized_out[i]['input_ids'] for i in range(len(tokenized_out))]
# input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype='long', truncating='post', padding='post')
# input_ids[0]

array([    2,  2431, 48502,  7997, 15775,  8028, 10855,     3,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0])

In [33]:
# len(input_ids)

143620

In [ ]:
# attention mask 에도 padding 을 주고 

# attention_masks = [tokenized_out[i]['attention_mask'] ]